# Animal Image Classification

# Imports

In [1]:
import os
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from PIL import Image, ImageFile

# Constants

In [2]:
PATH = "/Users/maukanmir/Downloads/animal_data"
BATCH_SIZE= 32
INPUT_SHAPE= (224,224,3)

# Functions

In [ ]:
def process_images(img_path, target_size):
  try:
    with Image.open(img_path) as image:
      image = image.resize()
  except IOError as e:
    print(f"There was an error processing this image: {e}")
    return None

# EDA Analysis